# Read in Data

In [48]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import ast
from scipy.spatial.distance import pdist

df = pd.read_csv("../../data/experiment_data/data_clean_with_elab.csv")
df.head() 

,Unnamed: 0,response_id,participant_id,response_text,response_date,condition,condition_order,item,response_chain,init_array,...,idea_minus_excluded5_txt,idea6_txt,idea_minus_excluded6_txt,source,trial_no,trial_no_excluded,elab_ibf,elab_ipf,elab_not_sw,elab_n_tokens
0,0,99ae32b8-e477-4d42-a98f-1a11116a8779,5c6d9a7d-79f2-491e-8592-5a4917313807,Dog chew toy,2023-07-26 02:15:58,f_l,3,shoe,6,"['5bc110cc-fea3-4728-bdba-e3a884bd63ff', 'ai_s...",...,Paint it to look like a face with the opening ...,Convey that you're selling drugs,Convey that you're selling drugs,r/artificial,19,19.0,2.463967,8.755670,3,3
1,1,632754e7-8297-4c55-91ec-485164c851f2,a062f627-4225-4634-9441-b6b41e0fe42f,Use the shoe as a ruler,2023-07-26 17:56:06,f_l,0,shoe,6,"['5d624ef0-ad38-441d-8982-86ad7da86fb1', '9d8b...",...,doorstop to hold open doors,Dog chew toy,Dog chew toy,r/singularity,20,20.0,2.954592,8.905649,3,6
2,2,052f291a-1f95-4154-b217-504009cc5127,5c6d9a7d-79f2-491e-8592-5a4917313807,Kicking object to release stress,2023-07-26 02:14:34,m_l,2,tire,6,"['2c277bc2-10b6-42f0-876c-f0a86bf0eaa6', 'ai_s...",...,yard art sculpture,diy ottoman or footstool,diy ottoman or footstool,r/artificial,19,18.0,3.809144,11.173407,4,5
3,3,312cc305-86b0-4d72-97ea-d5b551618376,a062f627-4225-4634-9441-b6b41e0fe42f,use it as a damper,2023-07-26 18:02:28,h,2,tire,6,"['353a7741-c733-4ca1-988c-a809c2b24df6', 'd108...",...,tire as an armchair or a shelf or as an exerci...,Carve the tire into a portrait of the first lady,Carve the tire into a portrait of the first lady,r/singularity,19,19.0,2.763950,6.175155,2,5
4,4,1d7fb44d-adba-4ac4-9360-4126b214c2ae,b904b2d5-0217-41bf-9741-5203f3f46e34,use a a toy for pets to hide in and explore,2023-07-26 16:46:37,m_l,3,tire,6,"['ai_seed1238', '052f291a-1f95-4154-b217-50400...",...,garden planter with flowers,fitness tool - tire flipping!,fitness tool - tire flipping!,Creative Mornings newsletter,20,19.0,4.684683,13.661652,5,11


# Get embeding of all ideas

In [58]:
model = SentenceTransformer('all-MiniLM-L6-v2')
example_cols = [x for x in df.columns if "_txt" in x and "excluded" not in x]

# Gather all ideas
df['init_array_list'] = df[example_cols].values.tolist()
all_ideas = []
all_ideas.extend(df['response_text'])
for x in example_cols:
    all_ideas.extend(df[x].to_list())
all_ideas = list(set(all_ideas))

# For convienence, create a dictionary mapping response_ids to response_text called rid2txt and txt2rid
df['init_array'] = df['init_array'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])
df_split = df['init_array'].apply(pd.Series)
df_split = df_split.rename(columns = lambda x : 'idea' + str(x + 1) + '_rid')
df = pd.concat([df[:], df_split[:]], axis=1)
rid_cols = [col for col in df.columns if "_rid" in col]
rid2txt = {df[rid_col][i]: df[txt_col][i] for i in range(len(df)) for rid_col, txt_col in zip(rid_cols, example_cols)}
txt2rid = {v: k for k, v in txt2rid.items()}

# Get embeddings and create idea2vec
embeddings = model.encode(all_ideas)
idea2vec = dict(zip(all_ideas, embeddings))

/var/folders/z1/zth478gs6r51th022yvszcbh0000gr/T/ipykernel_9225/3947677515.py:14: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_split = df['init_array'].apply(pd.Series)


# Compute semantic diversity metrics

In [55]:
from scipy.spatial.distance import pdist, cdist, euclidean
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def semantic_distances(response_text, example_list, idea2vec, txt2rid):
    ai_ideas = [x for x in example_list if txt2rid[x].startswith("ai_")]
    idea_vec = idea2vec[response_text]
    example_vecs = [idea2vec[example] for example in example_list]
    ai_example_vecs = [idea2vec[example] for example in ai_ideas]
    union_vecs = example_vecs + [idea_vec]

    distances = pdist(union_vecs, "euclidean")
    
    # distance between idea_vec and centroid of example_vecs
    centroid = np.mean(example_vecs, axis=0)
    cent_dist = euclidean(idea_vec, centroid)

    # max similarity and max distance for ai_example_vecs
    if ai_example_vecs:
        ai_sim = np.max(cosine_similarity([idea_vec], ai_example_vecs))
        ai_dist = np.max(cdist([idea_vec], ai_example_vecs, "euclidean"))
    else:
        ai_sim = 0
        ai_dist = 0

    measures = {
        'avg_pw_dist': np.mean(distances),
        'cent_dist': cent_dist,
        'ai_sim': ai_sim,
        'ai_dist': ai_dist
    }
    return measures


semantic_distances(df['response_text'][0], df['init_array_list'][0], idea2vec, txt2rid)


KeyError: 'hide cash in them'